In [2]:
import pandas as pd
import ipywidgets as widgets
import csv
from IPython.display import HTML


ElementDF = pd.read_csv('RuleElements.csv')
SelectorList = pd.read_csv('selectors.csv').columns.to_list()
OptionList = pd.read_csv('options.csv').columns.to_list()
CommonPredicates = ['agile','finesse','rage','unarmed','fear','mental','poison','disease','fear','sweep']
DamageTypes = ['','slashing','piercing','bludgeoning','precision','fire','cold','electricity','acid','poison',
               'sonic','force','mental','lawful','chaotic','good','evil','positive','negative']

def MakeRuleElement(Key,Options):
    ####################################################################
    # This takes in a key and options for that key and builds a rule element.
    #
    # key -- A string giving the key for the rule element.
    # options -- An array of strings containing the options for the rule element
    ####################################################################
    
    RuleElement = '{"key\":\"' + Key + '"'
    
    for Option in Options:
        RuleElement += ',' + Option
    
    RuleElement += '}'
    
    return print(RuleElement)

def GetPropertyList(row):
    ####################################################################
    # This returns a list of the applicable rule elements.
    #
    # To be used with df.apply(), this is a utility function and just
    # helps to store values.  It does not need to be run except to add 
    # new properties.
    ####################################################################
    
    row = row[PropertyList]
    
    passed = row[row == 'y'].index.tolist()
    return(passed)

def PreparePredicates(All, Any, Not):
    ####################################################################
    # This puts the predacate lists together into a single entity
    #
    # All -- List containing predicates which must all be satisfied
    # Any -- List containing predicates where one must be satified.
    # Not -- !All
    ####################################################################
    
    Predicate = str()
    
    if len(All + Any + Not) > 0:
        Predicate += '\"predicate\":{'
    
    if len(All) > 0:
        Predicate += '\"all\":[\"'
        i = 1
        for each in All:
            if i == len(All):
                Predicate += each + '\"'
            else:
                Predicate += each + '\",\"'
                i += 1
        Predicate += ']'
        
        if len(Any + Not) > 0:
            Predicate += ','
            
    if len(Any) > 0:
        Predicate += '\"any\":[\"'
        i = 1
        for each in Any:
            if i == len(Any):
                Predicate += each + '\"'
            else:
                Predicate += each + '\",\"'
                i += 1
        Predicate += ']'
        
        if len(Not) > 0:
            Predicate += ','
            
    if len(Not) > 0:
        Predicate += '\"not\":[\"'
        i = 1
        for each in Not:
            if i == len(Not):
                Predicate += each + '\"'
            else:
                Predicate += each + '\",\"'
                i += 1
        Predicate += ']'

    if len(All + Any + Not) > 0:
        Predicate += '}'
        
    return Predicate


def PrepareOptions(Selector, AdvancedSelector, Value, Type, DamageType, DieSize, DiceNumber, Predicate, Label, Name, Override):
    ####################################################################
    # This brings each option together and puts them into a single string.
    #
    # All inputs are strings with the exception of value, which is an integer.
    ####################################################################
    
    Options = []
    
    if len(Selector) > 0:
        Options += ['"selector":"' + AdvancedSelector + Selector + '"']
    
    if len(DamageType) > 0:
        if DamageType == 'precision':
            Options += ['"damageCategory":"' + DamageType + '"']
        else:
            Options += ['"damageType":"' + DamageType + '"']

    if len(DieSize) > 0:
        if Override:
            Options += ['"override":{"dieSize":"' + DieSize + '"}']
        else:
            Options += ['"dieSize":"' + DieSize + '"']
        
    if DiceNumber > 0:
        Options += ['"diceNumber":' + str(DiceNumber)]
    
    if abs(Value) > 0:
        Options += ['"value":' + str(Value)]
        
    if len(Type) > 0:
        Options += ['"type":"' + Type + '"']
        
    if len(Predicate) >0:
        Options += [Predicate]
    
    if Label:
        Options += ['"label":"YOUR TEXT HERE"']
    
    if Name:
        Options += ['"name":"YOUR TEXT HERE"']
        
    return Options

PropertyList = SelectorList
ElementDF['ValidSelectors'] = ElementDF.apply(GetPropertyList,axis = 1)

PropertyList = OptionList
ElementDF['ValidOptions'] = ElementDF.apply(GetPropertyList,axis = 1)

Key = widgets.Dropdown(
options=ElementDF.FriendlyName.tolist(),
value=ElementDF.FriendlyName.tolist()[0],
description='Key:',
disabled=False
)

All = widgets.SelectMultiple(
options=CommonPredicates,
value=[],
description='Predicate:All',
disabled=True )

Any = widgets.SelectMultiple(
options=CommonPredicates,
value=[],
description='Any',
disabled=True )

Not = widgets.SelectMultiple(
options=CommonPredicates,
value=[],
description='Not',
disabled=True )

Selector = widgets.Dropdown(
options=ElementDF.ValidSelectors[ElementDF.FriendlyName == Key.value].values[0],
value='all',
description='Selector',
disabled=True)

AdvancedSelector = widgets.Dropdown(
options=[('no advanced selector',''),('Only this item','{item|_id}-'),('swords','sword-weapon-group-'),('bows','bow-weapon-group-'),('knives','knife-weapon-group-'),('spears','spear-weapon-group-'),('axes','axe-weapon-group-'),('hammers','hammer-weapon-group-'),('picks','pick-weapon-group-'),('flails','flail-weapon-group-'),('brawling','brawling-weapon-group-'),('polearms','polearm-weapon-group-'),('sling','sling-weapon-group-'),('bombs','bomb-weapon-group-'),('shields','shield-weapon-group-'),('darts','dart-weapon-group-'),('clubs','clubs-weapon-group-')],
value='',
description='Advanced Selector',
disabled=True )

Label = widgets.Checkbox(
value=False,
description='Add a label?',
disabled=True,
indent=False
)

Name = widgets.Checkbox(
value=False,
description='Add a Name?',
disabled=True,
indent=False
)

DamageType = widgets.Dropdown(
options=DamageTypes,
value='',
description='Damage type',
disabled=True )

Value = widgets.IntText(
value=0,
description='Value:',
disabled=True)

DiceNumber = widgets.IntText(
value=0,
description='Number of dice:',
disabled=True )

DieSize = widgets.Dropdown(
options=['','d4','d6','d8','d10','d12'],
value='',
description='Die size:',
disabled=True )

Override = widgets.Checkbox(
value=False,
description='Override base damage?',
disabled=True,
indent=False )

Type = widgets.Dropdown(
options=['','circumstance','item','status'],
value='',
description='Type',
disabled=True )

UseAdvancedSelector = widgets.Checkbox(
value=False,
description = 'Use Advanced Selector?',
disabled=True
)

def ResetState(b):
    Selector.options = ElementDF.ValidSelectors[ElementDF.FriendlyName == Key.value].values[0]
    Selector.value = ElementDF.ValidSelectors[ElementDF.FriendlyName == Key.value].values[0][0]
    
    AdvancedSelector.value=''
    
    Selector.disabled=True
    AdvancedSelector.disabled=True
    
    DamageType.disabled=True
    Value.disabled=True
    DiceNumber.disabled=True
    DieSize.disabled=True
    Override.disabled=True
    Type.disabled=True
    All.disabled=True
    Any.disabled=True
    Not.disabled=True
    Label.disabled=True
    Name.disabled=True
    FinishButton.disabled=True
    
    Label.value=False
    Name.value=False
    DamageType.value=''
    Value.value=0
    DiceNumber.value=0
    DieSize.value=''
    Override.value=False
    Type.value=''
    UseAdvancedSelector.value=False
    
    return

def SetKey(b):
    
    ResetState(1)
    
    Selector.disabled=False
    AdvancedSelector.disabled=False
    SelectorButton.disabled=False
    
    return 

def SetSelector(b):
    
    Label.value=False
    Name.value=False
    DamageType.value=''
    Value.value=0
    DiceNumber.value=0
    DieSize.value=''
    Override.value=False
    Type.value=''
    UseAdvancedSelector.value=False
    
    if (('damage' in Selector.value) and (Key.value != 'Striking rune')):
        DamageType.disabled=False

    if 'value' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        Value.disabled=False

    if 'diceNumber' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        DiceNumber.disabled=False

    if 'dieSize' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        DieSize.disabled=False
        Override.disabled=False

    if 'type' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        Type.disabled=False

    if 'predicate' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        All.disabled=False
        Any.disabled=False
        Not.disabled=False

    Label.disabled=False
    Name.disabled=False
    
    FinishButton.disabled=False
    
    return

def FinishElement(b):

    Predicate = PreparePredicates(All.value, Any.value, Not.value)
    Options = PrepareOptions(Selector.value, AdvancedSelector.value, Value.value, Type.value, DamageType.value, DieSize.value, DiceNumber.value, Predicate, Label.value, Name.value, Override.value)

    MakeRuleElement(ElementDF.Key[ElementDF.FriendlyName == Key.value].values[0],Options)
    
    return

KeyButton = widgets.Button(description = "Select Key")
SelectorButton = widgets.Button(description = "Set Selectors", disabled=True)
FinishButton = widgets.Button(description = "Create Rule Element", disabled=True)
ResetButton = widgets.Button(description = "Reset")

display(Key)
display(KeyButton)

display(Selector)
display(AdvancedSelector)
display(SelectorButton)

display(DamageType)
display(Value)
display(DiceNumber)
display(DieSize)
display(Override)
display(Type)
display(All)
display(Any)
display(Not)
display(FinishButton)
display(ResetButton)

KeyButton.on_click(SetKey)
SelectorButton.on_click(SetSelector)
FinishButton.on_click(FinishElement)
ResetButton.on_click(ResetState)

Dropdown(description='Key:', options=('Flat Modifier', 'Damage Dice', 'Weapon Potency', 'Striking rune', 'Mult…

Button(description='Select Key', style=ButtonStyle())

Dropdown(description='Selector', disabled=True, options=('all', 'str-based', 'dex-based', 'con-based', 'int-ba…

Dropdown(description='Advanced Selector', disabled=True, options=(('no advanced selector', ''), ('Only this it…

Button(description='Set Selectors', disabled=True, style=ButtonStyle())

...

SelectMultiple(description='Predicate:All', disabled=True, options=('agile', 'finesse', 'rage', 'unarmed', 'fe…

SelectMultiple(description='Any', disabled=True, options=('agile', 'finesse', 'rage', 'unarmed', 'fear', 'ment…

SelectMultiple(description='Not', disabled=True, options=('agile', 'finesse', 'rage', 'unarmed', 'fear', 'ment…

Button(description='Create Rule Element', disabled=True, style=ButtonStyle())

Button(description='Reset', style=ButtonStyle())

In [3]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')